#1. Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
pkgs_list = c("googledrive", "PRROC","readxl", 
                "caret", "data.table",
              'readODS', 'stats','pROC')
new_pkgs = pkgs_list[!(pkgs_list %in% installed.packages()[,"Package"])]
if(length(new_pkgs)) install.packages(new_pkgs)

R[write to console]: Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘future.apply’, ‘progressr’, ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘gower’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘recipes’, ‘reshape2’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/listenv_0.8.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 36762 bytes (35 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to cons

In [4]:
%%R
library('caret')
library('PRROC')
library('readxl')
library('stats')
library('data.table')
library('readODS')
library('pROC')

R[write to console]: Loading required package: ggplot2

R[write to console]: Loading required package: lattice

R[write to console]: data.table 1.14.2 using 1 threads (see ?getDTthreads).  Latest news: r-datatable.com

R[write to console]: Type 'citation("pROC")' for a citation.

R[write to console]: 
Attaching package: ‘pROC’


R[write to console]: The following objects are masked from ‘package:stats’:

    cov, smooth, var




#2. Custom Functions for metrics

In [5]:
%%R
# Define Functions:
## F1- Score
# predicted: vector of predicted values
# expected: vector of observed value
# positive.class: class of binary predictions we are mostly interested in (e.g., "1", "0")

f1_score <- function(predicted, expected, positive.class) {
  
  # Generate Confusion Matrix
  c.matrix = as.matrix(table(expected, predicted))
  
  # Compute Precision
  precision <- diag(c.matrix) / colSums(c.matrix)
  
  # Compute Recall
  recall <- diag(c.matrix) / rowSums(c.matrix)
  
  # Compute F-1 Score
  f1 <-  ifelse(precision + recall == 0, 0, 2 * precision * recall / (precision + recall))
  
  # Extract F1-score for the pre-defined "positive class"
  f1 <- f1[positive.class]
  
  # Assuming that F1 is zero when it's not possible compute it
  f1[is.na(f1)] <- 0
  
  # Return F1-score
  return(f1)
}

## Balanced Accuracy (BACC)
# predicted: vector of predicted values
# expected: vector of observed value

balanced_accuracy <- function(predicted, expected) {
  
  # Generate Confusion Matrix
  c.matrix = as.matrix(table(predicted, expected))
  
  # First Row Generation
  first.row <- c.matrix[1,1] / (c.matrix[1,1] + c.matrix[1,2])  
  
  # Second Row Generation
  second.row <- c.matrix[2,2] / (c.matrix[2,1] + c.matrix[2,2])  
  
  # # "Balanced" proportion correct (you can use different weighting if needed)
  acc <- (first.row + second.row)/2 
  
  # Return Balanced Accuracy
  return(acc)
}

## Model Compare (Precision and False Discovery Rate [FDR])
# predicted: vector of predicted values
# expected: vector of observed value
# positive.class: class of binary predictions we are mostly interested in (e.g., "1", "0")

model_compare <- function(predicted, expected, positive.class) {
  
  # Generate Confusion Matrix
  c.matrix = as.matrix(table(expected, predicted))
  
  # Precision
  precision <- diag(c.matrix) / colSums(c.matrix)
  precision <- precision[positive.class]
  
  # False Discovery Rate
  fdr <- c.matrix[1,2]/(c.matrix[1,2] + c.matrix[2,2])
  
  # Return Precision and False Discovery Rate (FDR)
  return(c(precision, fdr))
}

#3. Load datasets

In [6]:
%%R
# Set dataset folder as working directory to load dataframes : 
wd = setwd("/content/drive/Shareddrives/Eye_Tracking_NN_SD/Image_processing/Publishable_material/Datasets_regressions/Wide_format")

In [7]:
%%R
# load data 
mydata2s = as.data.frame(read_ods("Dataset_regression_2s.ods"))
mydata5s = as.data.frame(read_ods("Dataset_regression_5s.ods"))
mydata10s = as.data.frame(read_ods("Dataset_regression_10s.ods"))
mydata15s= as.data.frame(read_ods("Dataset_regression_15s.ods"))
mydata15p=as.data.frame(read_ods("Dataset_regression_15p.ods"))
mydata30p=as.data.frame(read_ods("Dataset_regression_30p.ods"))
mydata50p=as.data.frame(read_ods("Dataset_regression_50p.ods"))
mydata80p=as.data.frame(read_ods("Dataset_regression_80p.ods"))
mydatafull=as.data.frame(read_ods("Dataset_regression_full.ods"))

In [8]:
%%R
# remove column 21 - R reads 'Intra-cell' like 'Intracell'. We remove 'Intra-cell' column.
mydata2s = mydata2s[,-21]
mydata5s = mydata5s[,-21]
mydata10s = mydata10s[,-21]
mydata15s = mydata15s[,-21]
mydata15p = mydata15p[,-21]
mydata30p = mydata30p[,-21]
mydata50p = mydata50p[,-21]
mydata80p = mydata80p[,-21]
mydatafull = mydatafull[,-21]

#4. Fit and test Logit model with each dataset 

In [9]:
%%R
setwd('/content/drive/Shareddrives/Eye_Tracking_NN_SD/Image_processing/Publishable_material/Models_performance/Logit/ROC_EQ')

In [10]:
%%R
#### 2 seconds ####
fit_data <- mydata2s[which(mydata2s$data_collection=="first"),]
eval_data <- mydata2s[which(mydata2s$data_collection=="second"),]
## Model fit:
logit_model_all <- glm(Eq ~ Own + Other + Intracell, family=binomial(link="logit"), data=fit_data)
# Predictive ability:
fitted.results = predict(logit_model_all, newdata=subset(eval_data, select=c("Own", "Other", "Intracell")), type='response')
fitted.results = ifelse(fitted.results > 0.5,1,0)
misClasificError <- mean(fitted.results != eval_data$Eq)
accuracy=1-misClasificError
# Get ROC and AUC:
fg <- fitted.results[eval_data$Eq==1] 
bg <- fitted.results[eval_data$Eq==0]
roc <- roc.curve(scores.class0 = fg,
                 scores.class1 = bg,
                 curve = T)
pr <- pr.curve(scores.class0 = fg,
               scores.class1 = bg, 
               curve = T)
f1 <- f1_score(fitted.results,
               eval_data$Eq,
               positive.class="1")
balanced_accuracy_full <- balanced_accuracy(fitted.results, eval_data$Eq)
ft = as.factor(fitted.results)
true_y = as.factor(eval_data$Eq)
recall <- sensitivity(ft, true_y, positive="1")
outcome_2s <- cbind(accuracy, roc$auc,
                    pr$auc.integral,
                    recall,
                    f1,
                    balanced_accuracy_full)
colnames(outcome_2s) <- c("ACCURACY", "AUC", "PR", "Recall", "f1-score", "BACC")
print(outcome_2s)
roc = roc(eval_data$Eq, fitted.results)
png(file="roc_2s.png",width=600, height=350)
plot.roc(roc)
dev.off()
rm(fit_data, eval_data, fitted.results, roc)

   ACCURACY       AUC        PR    Recall  f1-score      BACC
1 0.6176471 0.6224636 0.5610644 0.7169421 0.6402214 0.6260714


R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



In [11]:
%%R
#### 5 seconds ####

fit_data <- mydata5s[which(mydata5s$data_collection=="first"),]
eval_data <- mydata5s[which(mydata5s$data_collection=="second"),]
# Model fit:
logit_model_all <- glm(Eq ~ Own + Other + Intracell, family=binomial(link="logit"), data=fit_data)
# Predictive ability:
fitted.results = predict(logit_model_all, newdata=subset(eval_data, select=c("Own", "Other", "Intracell")), type='response')
fitted.results = ifelse(fitted.results > 0.5,1,0)
misClasificError <- mean(fitted.results != eval_data$Eq)
accuracy=1-misClasificError
# Get ROC and AUC:
fg <- fitted.results[eval_data$Eq==1] 
bg <- fitted.results[eval_data$Eq==0]
roc <- roc.curve(scores.class0 = fg,
                 scores.class1 = bg,
                 curve = T)
pr <- pr.curve(scores.class0 = fg,
               scores.class1 = bg, 
               curve = T)
f1 <- f1_score(fitted.results,
               eval_data$Eq,
               positive.class="1")
balanced_accuracy_full <- balanced_accuracy(fitted.results, eval_data$Eq)
ft = as.factor(fitted.results)
true_y = as.factor(eval_data$Eq)
recall <- sensitivity(ft, true_y, positive="1")
outcome_5s <- cbind(accuracy, roc$auc,
                    pr$auc.integral,
                    recall,
                    f1,
                    balanced_accuracy_full)
colnames(outcome_5s) <- c("ACCURACY", "AUC", "PR", "Recall", "f1-score", "BACC")
print(outcome_5s)
roc = roc(eval_data$Eq, fitted.results)
png(file="roc_5s.png",width=600, height=350)
plot.roc(roc)
dev.off()
rm(fit_data, eval_data, fitted.results, roc)

   ACCURACY       AUC        PR    Recall  f1-score      BACC
1 0.6392157 0.6440884 0.5779798 0.7396694 0.6605166 0.6483333


R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



In [12]:
%%R
#### 10 seconds ####

fit_data <- mydata10s[which(mydata10s$data_collection=="first"),]
eval_data <- mydata10s[which(mydata10s$data_collection=="second"),]
# Model fit:
logit_model_all <- glm(Eq ~ Own + Other + Intracell, family=binomial(link="logit"), data=fit_data)
# Predictive ability:
fitted.results = predict(logit_model_all, newdata=subset(eval_data, select=c("Own", "Other", "Intracell")), type='response')
fitted.results = ifelse(fitted.results > 0.5,1,0)
misClasificError <- mean(fitted.results != eval_data$Eq)
accuracy=1-misClasificError
# Get ROC and AUC:
fg <- fitted.results[eval_data$Eq==1] 
bg <- fitted.results[eval_data$Eq==0]
roc <- roc.curve(scores.class0 = fg,
                 scores.class1 = bg,
                 curve = T)
pr <- pr.curve(scores.class0 = fg,
               scores.class1 = bg, 
               curve = T)
f1 <- f1_score(fitted.results,
               eval_data$Eq,
               positive.class="1")
balanced_accuracy_full <- balanced_accuracy(fitted.results, eval_data$Eq)
ft = as.factor(fitted.results)
true_y = as.factor(eval_data$Eq)
recall <- sensitivity(ft, true_y, positive="1")
outcome_10s <- cbind(accuracy, roc$auc,
                    pr$auc.integral,
                    recall,
                    f1,
                    balanced_accuracy_full)
colnames(outcome_10s) <- c("ACCURACY", "AUC", "PR", "Recall", "f1-score", "BACC")
print(outcome_10s)
roc = roc(eval_data$Eq, fitted.results)
png(file="roc_10s.png",width=600, height=350)
plot.roc(roc)
dev.off()
rm(fit_data, eval_data, fitted.results, roc)

   ACCURACY       AUC        PR    Recall f1-score      BACC
1 0.6578431 0.6631152 0.5925096 0.7665289  0.68011 0.6687974


R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



In [13]:
%%R
#### 15 seconds ####

fit_data <- mydata15s[which(mydata10s$data_collection=="first"),]
eval_data <- mydata15s[which(mydata10s$data_collection=="second"),]
# Model fit:
logit_model_all <- glm(Eq ~ Own + Other + Intracell, family=binomial(link="logit"), data=fit_data)
# Predictive ability:
fitted.results = predict(logit_model_all, newdata=subset(eval_data, select=c("Own", "Other", "Intracell")), type='response')
fitted.results = ifelse(fitted.results > 0.5,1,0)
misClasificError <- mean(fitted.results != eval_data$Eq)
accuracy=1-misClasificError
# Get ROC and AUC:
fg <- fitted.results[eval_data$Eq==1] 
bg <- fitted.results[eval_data$Eq==0]
roc <- roc.curve(scores.class0 = fg,
                 scores.class1 = bg,
                 curve = T)
pr <- pr.curve(scores.class0 = fg,
               scores.class1 = bg, 
               curve = T)
f1 <- f1_score(fitted.results,
               eval_data$Eq,
               positive.class="1")
balanced_accuracy_full <- balanced_accuracy(fitted.results, eval_data$Eq)
ft = as.factor(fitted.results)
true_y = as.factor(eval_data$Eq)
recall <- sensitivity(ft, true_y, positive="1")
outcome_15s <- cbind(accuracy, roc$auc,
                    pr$auc.integral,
                    recall,
                    f1,
                    balanced_accuracy_full)
colnames(outcome_15s) <- c("ACCURACY", "AUC", "PR", "Recall", "f1-score", "BACC")
print(outcome_15s)
roc = roc(eval_data$Eq, fitted.results)
png(file="roc_15s.png",width=600, height=350)
plot.roc(roc)
dev.off()
rm(fit_data, eval_data, fitted.results, roc)

   ACCURACY       AUC        PR    Recall  f1-score      BACC
1 0.6637255 0.6693136 0.5968645 0.7789256 0.6873291 0.6760544


R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



In [16]:
%%R
#### 15 percent ####
# Model fit:
fit_data <- mydata15p[which(mydata15p$data_collection=="first"),]
eval_data <- mydata15p[which(mydata15p$data_collection=="second"),]
logit_model_all <- glm(Eq ~ Own + Other + Intracell, family=binomial(link="logit"), data=fit_data)
summary(logit_model_all)
# Predictive ability:
fitted.results = predict(logit_model_all, newdata=subset(eval_data, select=c("Own", "Other", "Intracell")), type='response')
fitted.results = ifelse(fitted.results > 0.5,1,0)
misClasificError <- mean(fitted.results != eval_data$Eq)
accuracy=1-misClasificError
# Get ROC and AUC:
eval_data_Eq = eval_data$Eq[!is.na(fitted.results)]
fitted.results_noNa = fitted.results[!is.na(fitted.results)]
fg <- fitted.results_noNa[eval_data_Eq==1] 
bg <- fitted.results_noNa[eval_data_Eq==0]
fg = fg[!is.na(fg)]
bg = bg[!is.na(bg)]
roc <- roc.curve(scores.class0 = fg,
                 scores.class1 = bg,
                 curve = T)
pr <- pr.curve(scores.class0 = fg,
               scores.class1 = bg, 
               curve = T)
f1 <- f1_score(fitted.results_noNa,
               eval_data_Eq,
               positive.class="1")
balanced_accuracy_full <- balanced_accuracy(fitted.results, eval_data$Eq)
ft = as.factor(fitted.results)
true_y = as.factor(eval_data$Eq)
recall <- sensitivity(ft, true_y, positive="1")
outcome_15p <- cbind(accuracy, roc$auc,
                    pr$auc.integral,
                    recall,
                    f1,
                    balanced_accuracy_full)
colnames(outcome_15p) <- c("ACCURACY", "AUC", "PR", "Recall", "f1-score", "BACC")
print(outcome_15p)
roc = roc(eval_data$Eq, fitted.results)
png(file="roc_15p.png",width=600, height=350)
plot.roc(roc)
dev.off()
rm(train_data, test_data, fitted.results, roc)

  ACCURACY       AUC        PR   Recall  f1-score      BACC
1       NA 0.6041149 0.5509571 0.625523 0.5985986 0.6039216


R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



In [17]:
%%R
#### 30 percent ####

fit_data <- mydata30p[which(mydata10s$data_collection=="first"),]
eval_data <- mydata30p[which(mydata10s$data_collection=="second"),]
# Model fit:
logit_model_all <- glm(Eq ~ Own + Other + Intracell, family=binomial(link="logit"), data=fit_data)
# Predictive ability:
fitted.results = predict(logit_model_all, newdata=subset(eval_data, select=c("Own", "Other", "Intracell")), type='response')
fitted.results = ifelse(fitted.results > 0.5,1,0)
misClasificError <- mean(fitted.results != eval_data$Eq)
accuracy=1-misClasificError
# Get ROC and AUC:
fg <- fitted.results[eval_data$Eq==1] 
bg <- fitted.results[eval_data$Eq==0]
roc <- roc.curve(scores.class0 = fg,
                 scores.class1 = bg,
                 curve = T)
pr <- pr.curve(scores.class0 = fg,
               scores.class1 = bg, 
               curve = T)
f1 <- f1_score(fitted.results,
               eval_data$Eq,
               positive.class="1")
balanced_accuracy_full <- balanced_accuracy(fitted.results, eval_data$Eq)
ft = as.factor(fitted.results)
true_y = as.factor(eval_data$Eq)
recall <- sensitivity(ft, true_y, positive="1")
outcome_30p <- cbind(accuracy, roc$auc,
                    pr$auc.integral,
                    recall,
                    f1,
                    balanced_accuracy_full)
colnames(outcome_30p) <- c("ACCURACY", "AUC", "PR", "Recall", "f1-score", "BACC")
print(outcome_30p)
roc = roc(eval_data$Eq, fitted.results)
png(file="roc_30p.png",width=600, height=350)
plot.roc(roc)
dev.off()
rm(fit_data, eval_data, fitted.results, roc)

   ACCURACY       AUC        PR    Recall  f1-score      BACC
1 0.6254902 0.6282225 0.5691078 0.6818182 0.6333973 0.6290323


R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



In [18]:
%%R
#### 50 percent ####
#
fit_data <- mydata50p[which(mydata10s$data_collection=="first"),]
eval_data <- mydata50p[which(mydata10s$data_collection=="second"),]
# Model fit:
logit_model_all <- glm(Eq ~ Own + Other + Intracell, family=binomial(link="logit"), data=fit_data)
# Predictive ability:
fitted.results = predict(logit_model_all, newdata=subset(eval_data, select=c("Own", "Other", "Intracell")), type='response')
fitted.results = ifelse(fitted.results > 0.5,1,0)
misClasificError <- mean(fitted.results != eval_data$Eq)
accuracy=1-misClasificError
# Get ROC and AUC:
fg <- fitted.results[eval_data$Eq==1] 
bg <- fitted.results[eval_data$Eq==0]
roc <- roc.curve(scores.class0 = fg,
                 scores.class1 = bg,
                 curve = T)
pr <- pr.curve(scores.class0 = fg,
               scores.class1 = bg, 
               curve = T)
f1 <- f1_score(fitted.results,
               eval_data$Eq,
               positive.class="1")
balanced_accuracy_full <- balanced_accuracy(fitted.results, eval_data$Eq)
ft = as.factor(fitted.results)
true_y = as.factor(eval_data$Eq)
recall <- sensitivity(ft, true_y, positive="1")
outcome_50p <- cbind(accuracy, roc$auc,
                    pr$auc.integral,
                    recall,
                    f1,
                    balanced_accuracy_full)
colnames(outcome_50p) <- c("ACCURACY", "AUC", "PR", "Recall", "f1-score", "BACC")
print(outcome_50p)
roc = roc(eval_data$Eq, fitted.results)
png(file="roc_50p.png",width=600, height=350)
plot.roc(roc)
dev.off()
rm(fit_data, eval_data, fitted.results, roc)

   ACCURACY       AUC        PR    Recall  f1-score      BACC
1 0.6509804 0.6556834 0.5876654 0.7479339 0.6703704 0.6598234


R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



In [19]:
%%R
#### 80 percent ####

fit_data <- mydata80p[which(mydata10s$data_collection=="first"),]
eval_data <- mydata80p[which(mydata10s$data_collection=="second"),]
# Model fit:
logit_model_all <- glm(Eq ~ Own + Other + Intracell, family=binomial(link="logit"), data=fit_data)
# Predictive ability:
fitted.results = predict(logit_model_all, newdata=subset(eval_data, select=c("Own", "Other", "Intracell")), type='response')
fitted.results = ifelse(fitted.results > 0.5,1,0)
misClasificError <- mean(fitted.results != eval_data$Eq)
accuracy=1-misClasificError
# Get ROC and AUC:
fg <- fitted.results[eval_data$Eq==1] 
bg <- fitted.results[eval_data$Eq==0]
roc <- roc.curve(scores.class0 = fg,
                 scores.class1 = bg,
                 curve = T)
pr <- pr.curve(scores.class0 = fg,
               scores.class1 = bg, 
               curve = T)
f1 <- f1_score(fitted.results,
               eval_data$Eq,
               positive.class="1")
balanced_accuracy_full <- balanced_accuracy(fitted.results, eval_data$Eq)
ft = as.factor(fitted.results)
true_y = as.factor(eval_data$Eq)
recall <- sensitivity(ft, true_y, positive="1")
outcome_80p <- cbind(accuracy, roc$auc,
                    pr$auc.integral,
                    recall,
                    f1,
                    balanced_accuracy_full)
colnames(outcome_80p) <- c("ACCURACY", "AUC", "PR", "Recall", "f1-score", "BACC")
print(outcome_80p)
roc = roc(eval_data$Eq, fitted.results)
png(file="roc_80p.png",width=600, height=350)
plot.roc(roc)
dev.off()
rm(fit_data, eval_data, fitted.results, roc)

   ACCURACY       AUC        PR    Recall f1-score     BACC
1 0.6617647 0.6682497 0.5940674 0.7954545 0.690583 0.677822


R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



In [20]:
%%R
#### FULL ####
# Model fit:

fit_data <- mydatafull[which(mydata10s$data_collection=="first"),]
eval_data <- mydatafull[which(mydata10s$data_collection=="second"),]
# Model fit:
logit_model_all <- glm(Eq ~ Own + Other + Intracell, family=binomial(link="logit"), data=fit_data)
# Predictive ability:
fitted.results = predict(logit_model_all, newdata=subset(eval_data, select=c("Own", "Other", "Intracell")), type='response')
fitted.results = ifelse(fitted.results > 0.5,1,0)
misClasificError <- mean(fitted.results != eval_data$Eq)
accuracy=1-misClasificError
# Get ROC and AUC:
fg <- fitted.results[eval_data$Eq==1] 
bg <- fitted.results[eval_data$Eq==0]
roc <- roc.curve(scores.class0 = fg,
                 scores.class1 = bg,
                 curve = T)
pr <- pr.curve(scores.class0 = fg,
               scores.class1 = bg, 
               curve = T)
f1 <- f1_score(fitted.results,
               eval_data$Eq,
               positive.class="1")
balanced_accuracy_full <- balanced_accuracy(fitted.results, eval_data$Eq)
ft = as.factor(fitted.results)
true_y = as.factor(eval_data$Eq)
recall <- sensitivity(ft, true_y, positive="1")
outcome_full <- cbind(accuracy, roc$auc,
                    pr$auc.integral,
                    recall,
                    f1,
                    balanced_accuracy_full)
colnames(outcome_full) <- c("ACCURACY", "AUC", "PR", "Recall", "f1-score", "BACC")
print(outcome_full)
roc = roc(eval_data$Eq, fitted.results)
png(file="roc_full.png",width=600, height=350)
plot.roc(roc)
dev.off()
rm(fit_data, eval_data, fitted.results, roc)

   ACCURACY       AUC        PR    Recall f1-score      BACC
1 0.6735294 0.6796441 0.6040937 0.7995868 0.699187 0.6884277


R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



#5. Save perfomance results

In [21]:
%%R
#### save results ####
setwd('/content/drive/Shareddrives/Eye_Tracking_NN_SD/Image_processing/Publishable_material/Models_performance/Logit')
outcome_timing = rbind(outcome_full, outcome_15s, outcome_10s, outcome_5s, outcome_2s)

colnames(outcome_timing ) <- c("ACCURACY", "AUC", "PR", "Recall", "f1-score", "BACC")

outcome_timing <- as.data.frame(outcome_timing)
Dataset1 = c ('full', '15s', '10s', '5s', '2s')
outcome_timing = as.data.frame(cbind(Dataset1, outcome_timing))
write.csv(outcome_timing, file = "outcome_logit_timing.csv")


outcome_percentage = rbind(outcome_full, outcome_80p, outcome_50p, outcome_30p, outcome_15p)

colnames(outcome_percentage ) <- c("ACCURACY", "AUC", "PR", "Recall", "f1-score", "BACC")

outcome_percentage <- as.data.frame(outcome_percentage)
Dataset2 = c ('full', '80', '50', '30', '15' )
outcome_percentage = as.data.frame(cbind(Dataset2, outcome_percentage))
write.csv(outcome_percentage , file = "outcome_logit_percentage.csv")

In [22]:
%%R
outcome_timing

     Dataset1  ACCURACY       AUC        PR    Recall  f1-score      BACC
X1       full 0.6735294 0.6796441 0.6040937 0.7995868 0.6991870 0.6884277
X1.1      15s 0.6637255 0.6693136 0.5968645 0.7789256 0.6873291 0.6760544
X1.2      10s 0.6578431 0.6631152 0.5925096 0.7665289 0.6801100 0.6687974
X1.3       5s 0.6392157 0.6440884 0.5779798 0.7396694 0.6605166 0.6483333
X1.4       2s 0.6176471 0.6224636 0.5610644 0.7169421 0.6402214 0.6260714


In [23]:
%%R
outcome_percentage

     Dataset2  ACCURACY       AUC        PR    Recall  f1-score      BACC
X1       full 0.6735294 0.6796441 0.6040937 0.7995868 0.6991870 0.6884277
X1.1       80 0.6617647 0.6682497 0.5940674 0.7954545 0.6905830 0.6778220
X1.2       50 0.6509804 0.6556834 0.5876654 0.7479339 0.6703704 0.6598234
X1.3       30 0.6254902 0.6282225 0.5691078 0.6818182 0.6333973 0.6290323
X1.4       15        NA 0.6041149 0.5509571 0.6255230 0.5985986 0.6039216
